In [1]:
%cd ..

/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_Drounding_DINO


/home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_Drounding_DINO/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import glob
import pandas as pd
from utils import AnnotationDF
import cv2
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import torch
from metrics import COCO_annotation_loader, count_metrics

In [47]:
CLASSES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 
    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 
    'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [4]:
def get_IoU(bbox1, bbox2):
        intersection_h_max = min(bbox1[1] + (bbox1[3]/2), bbox2[1] + (bbox2[3]/2))
        intersection_h_min = max(bbox1[1] - (bbox1[3]/2), bbox2[1] - (bbox2[3]/2))
        intersection_w_max = min(bbox1[0] + (bbox1[2]/2), bbox2[0] + (bbox2[2]/2))
        intersection_w_min = max(bbox1[0] - (bbox1[2]/2), bbox2[0] - (bbox2[2]/2))   
        intersection_area = (intersection_h_max - intersection_h_min) * (intersection_w_max - intersection_w_min)
        union = bbox1[2]*bbox1[3] + bbox2[2]*bbox2[3] - intersection_area
        return round(intersection_area / union, 3)

In [26]:
class pandas_annotation_loader():
    def __init__(self, ann_file, filters=None):
        data = AnnotationDF()
        data.load_annotation(ann_file)
        self.df = data.df.copy()
        self.filenames = sorted(self.df.image_name.unique())
        self.filters = filters


    def __len__(self):
        return len(self.df.shape[0])
    
    

    def __get_from_dataframe(self, filename):
        data = self.df[self.df.image_name == filename].sort_values(by=['probability'], ascending=False)

        # Filter
        if self.filters is not None:
            for filter in self.filters:
                data = data[filter[0](data, **filter[1])]

        # classes
        class_indexes = data.loc[:,['class_index']].to_numpy()
        # confidence
        confidences = data.loc[:,['probability']].to_numpy()
        # BBOX in format xywh
        bbox_string2bbox = lambda line: np.array([float(x) for x in line[0].split(' ')])
        bboxes = np.array(list(map(bbox_string2bbox, data.loc[:,['bbox']].to_numpy())))
        if bboxes.shape[0] > 0:
            return np.concatenate((bboxes, confidences, class_indexes), axis=1)
        else: return np.array([])

    # Возвращает list[Detection] по индексу
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        return filename, self.__get_from_dataframe(filename)
    
    # Возвращает list[Detection] по индексу
    def get_annotation_by_name(self, filename):
        if filename not in self.filenames:
            print('not in the list')
            return None
        else:
            return self.__get_from_dataframe(filename) 

In [32]:
def nms_filter(detections, iou_threshold):
    # BBOX in format xywh
    bbox_string2bbox = lambda line: np.array([float(x) for x in line.split(' ')])
    keep_list = []
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        for sample in keep_list:
            if get_IoU(bbox_string2bbox(sample.bbox), 
                       bbox_string2bbox(detection.bbox) ) > iou_threshold:
                keep = False
                break
        keep_list.append(detection)
        mask.append(keep)
    return np.array(mask)

def nms_filter_one_class(detections, iou_threshold):
    # BBOX in format xywh
    bbox_string2bbox = lambda line: np.array([float(x) for x in line.split(' ')])
    keep_list = []
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        for sample in keep_list:
            if get_IoU(bbox_string2bbox(sample.bbox), 
                       bbox_string2bbox(detection.bbox) ) > iou_threshold and (sample.class_index == detection.class_index):
                keep = False
                break
        keep_list.append(detection)
        mask.append(keep)
    return np.array(mask)

def dumb_probability_filter(detections, probability_threshold):
    mask = []
    for _, detection in detections.iterrows():
        keep = True
        if detection.probability <= probability_threshold:
            keep = False
        mask.append(keep)
    return np.array(mask)


nms_filter_args = {'iou_threshold':0.7}
nms_filter_one_class_args = {'iou_threshold':0.6}
dumb_probability_filter_args = {'probability_threshold':0.4}

filters = [
    (nms_filter, nms_filter_args),
    (nms_filter_one_class, nms_filter_one_class_args),
    (dumb_probability_filter, dumb_probability_filter_args)
]

In [53]:
ann_loader = COCO_annotation_loader('data/labels_train2007')
det_loader = pandas_annotation_loader('data/VOC2007/train2007/df_annotations', filters)

ap, ap_class = count_metrics(det_loader, ann_loader)

TypeError: 'numpy.float64' object is not callable

In [49]:
data = AnnotationDF(CLASSES)
data.load_annotation('data/VOC2007/train2007/df_annotations')

In [52]:
data.df[data.df.image_name == '000026']

,image_name,class,class_index,num_boxes,probability,bbox,box_area
252,000026,aeroplane,0,24,0.397380,0.42145922780036926 0.5076552629470825 0.49464...,0.135783
253,000026,bicycle,1,24,0.397827,0.42110782861709595 0.5064442157745361 0.49437...,0.134620
254,000026,bird,2,24,0.303866,0.4214317500591278 0.506939172744751 0.4954860...,0.136462
255,000026,boat,3,24,0.236970,0.421335369348526 0.5069716572761536 0.4947617...,0.136300
256,000026,bottle,4,24,0.146020,0.420897901058197 0.5076733231544495 0.4940853...,0.135332
257,000026,bus,5,24,0.430206,0.42121586203575134 0.506991982460022 0.494358...,0.134922
258,000026,car,6,24,0.908009,0.4213421940803528 0.5073727965354919 0.495557...,0.136391
259,000026,cat,7,24,0.228420,0.42106500267982483 0.5076236128807068 0.49438...,0.135584
260,000026,chair,8,24,0.447504,0.42123666405677795 0.5068941712379456 0.49416...,0.134190
261,000026,chair,8,24,0.148393,0.433524489402771 0.42933180928230286 0.061866...,0.003381


In [34]:
map50, map = ap[:, 0].mean(), ap.mean(1).mean()
print(map50, map)

0.7723831290606439 0.6606398756647319


In [42]:
map50, map = ap[:, 0].mean(), ap.mean(1).mean()
print(map50, map)

0.7723831290606439 0.6606398756647319


In [45]:
for name, ap_cl in zip (CLASSES, ap[:,0]):
    print(f'{name} : {ap_cl}')

aeroplane : 0.9390099056133667
bicycle : 0.7824581813167448
bird : 0.89878016333406
boat : 0.7724062272923456
bottle : 0.6335198812422658
bus : 0.8405905760597341
car : 0.836987356245467
cat : 0.8713239033883763
chair : 0.5010853885568489
cow : 0.8462645531731038
diningtable : 0.5867443275606313
dog : 0.83988269530333
horse : 0.9172434483094108
motorbike : 0.7526368790663791
person : 0.8489077255604253
pottedplant : 0.6862477413501388
sheep : 0.7140282672867289
sofa : 0.5433652536596714
train : 0.853910213685901
tvmonitor : 0.7822698932079473


In [15]:
for name, ap_cl in zip (CLASSES, ap[:,-1]):
    print(f'{name} : {ap_cl}')

aeroplane : 0.21254799343939446
bicycle : 0.19671853887388674
bird : 0.19527639964760754
boat : 0.087011846423598
bottle : 0.09471990778151669
bus : 0.28263913187403433
car : 0.17212190791643567
cat : 0.2794418293701749
chair : 0.06071717194393426
cow : 0.15184847838808918
diningtable : 0.16099513885638356
dog : 0.26943889903029705
horse : 0.26480642772761154
motorbike : 0.14732393725706605
person : 0.152191872892028
pottedplant : 0.025829802010285303
sheep : 0.1073687360217109
sofa : 0.20218703670607296
train : 0.19873540567009604
tvmonitor : 0.09773978572894092


In [95]:
N = 3
preds = torch.zeros((N, 6))
preds[:,5] = torch.randint(7, (N,))
preds

tensor([[0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 6.],
        [0., 0., 0., 0., 0., 0.]])

In [96]:
M = 2
labels = torch.zeros((M, 5))
labels[:,0] = torch.randint(7, (M,))
labels

tensor([[5., 0., 0., 0., 0.],
        [5., 0., 0., 0., 0.]])

In [98]:
a = torch.arange(16).reshape(-1,4).unsqueeze(1).chunk(2,2)[0]
print(a.shape)
# a

torch.Size([4, 1, 2])


In [99]:
b = torch.arange(20).reshape(-1,4).unsqueeze(0).chunk(2,2)[1]
print(b.shape)
# b

torch.Size([1, 5, 2])


In [102]:
torch.Tensor([[1],[2],[1]]) == torch.Tensor([[1,1,1,1]])

tensor([[ True,  True,  True,  True],
        [False, False, False, False],
        [ True,  True,  True,  True]])

In [93]:
a = torch.Tensor([[1,2],
                  [3,4]])
b = torch.Tensor([[3,3],
                  [4,4]])
torch.where(a==2)

(tensor([0]), tensor([1]))

In [108]:
torch.stack((torch.Tensor([1,2,3]), torch.Tensor([4,5,6])  ), 1)

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

In [113]:
torch.cat((torch.Tensor([1,2,3]), torch.Tensor([1,2,3])),0)

tensor([1., 2., 3., 1., 2., 3.])